# Sequential Decision Modeling

## Step 1: Narrative

## Step 2: Identify Core Metrics

* Maximize contribution.
* Demand for each day is unknown.
    * $\hat{W}_{t+1}=\hat{D}_{t+1}$: Demand for today.
    * $\hat{D} \sim \mathcal{N}(\mu = 50, \sigma^2 = 5)$
* Decision variable $x_t$: How many pounds of bacon we'll order at the end of the day. $t$
* Constants:
    * $p=25$
    * $c=15$
* Initial values:
    * $R_0=10$
* Policies:
    * $\theta \in [40,60]$

## Step 3: Mathematical Model

### State:

$$S_t=(R_t, \hat{D}_{t+1}, c, p, \mu, \sigma^2)$$

### Decision Variables:

$$X^\pi = \max(\theta - R_t, 0)$$

### Exogenous Informartion:

$$\hat{D}_{t+1} \sim \mathcal{N}(\mu = 50, \sigma^2 = 5)$$

### Transition Function:

$$S^M=\max(R_t+x_t-\hat{D}_{t+1}, 0)$$

### Objective Function:

$$\max_{\pi} \sum_{t=1}^T C(S_t,x_t) | S_0$$

$$C(S_t,x_t)=-c(x_t)+p(\min(R_t+x_t,\hat{D}_{t+1}))$$

## Step 4: Uncertainity Model

* Previously defined...

## Step 5: Designing Policies

$$\theta \in [40,60]$$

## Step 6: Evaluating Policies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def X_t(r_t: int, theta: int) -> int:
    return np.max([0, theta - r_t])

In [3]:
c = 15
p = 25
mu = 50
var = 5
r_0 = 10

theta_space = list(range(40, 61))

for theta in theta_space:
    contributions = []

    for _ in range(1000):
        scenarios = []
        r_t = r_0
    
        for i in range(1, 11):
            # Buy inventory
            x_t = X_t(r_t, theta)

            # Exogenous information
            D_t_1 = np.round(np.random.normal(mu, np.sqrt(var)))

            # Transition function
            r_t_1 = np.max([x_t + r_t - D_t_1, 0])

            contribution = -c * x_t + p * (np.min([x_t + r_t, D_t_1]))

            scenarios.append({
                "t": i,
                "r_t": r_t,
                "x_t": x_t,
                "D_t_1": D_t_1,
                "C": contribution,
            })

            r_t = r_t_1

        theta_df = pd.DataFrame.from_records(scenarios)

        contributions.append(theta_df.C.sum())

    print(f"Theta = {theta} -> {np.mean(contributions):.2f}")

Theta = 40 -> 4150.00
Theta = 41 -> 4250.00
Theta = 42 -> 4349.98
Theta = 43 -> 4449.86
Theta = 44 -> 4549.70
Theta = 45 -> 4648.86
Theta = 46 -> 4746.20
Theta = 47 -> 4839.91
Theta = 48 -> 4923.74
Theta = 49 -> 4995.72
Theta = 50 -> 5044.78
Theta = 51 -> 5079.06
Theta = 52 -> 5099.70
Theta = 53 -> 5090.73
Theta = 54 -> 5082.55
Theta = 55 -> 5067.95
Theta = 56 -> 5061.55
Theta = 57 -> 5047.18
Theta = 58 -> 5027.91
Theta = 59 -> 5017.35
Theta = 60 -> 5005.89


In [4]:
import yfinance as yf

In [5]:
ticker = "AAPL"
df = yf.download(ticker, period='5y', interval='1d')['Close']
df['future_price'] = df['AAPL'].pct_change(periods=5).shift(-5)
df.head(10)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Ticker,AAPL,future_price
Date,,
2020-10-28,108.166199,0.033723
2020-10-29,112.173820,0.032171
2020-10-30,105.890068,0.092180
2020-11-02,105.802498,0.071258
2020-11-03,107.426949,0.051884
2020-11-04,111.813911,0.041289
2020-11-05,115.782585,0.003240
2020-11-06,115.651047,0.004803
2020-11-09,113.341743,0.034216


In [6]:
df['final_signal'] = 0
alpha = 0.02
df.loc[df['future_price'] > alpha, 'final_signal'] = 1
df.loc[df['future_price'] < -alpha, 'final_signal'] = -1
df

Ticker,AAPL,future_price,final_signal
Date,,,
2020-10-28,108.166199,0.033723,1
2020-10-29,112.173820,0.032171,1
2020-10-30,105.890068,0.092180,1
2020-11-02,105.802498,0.071258,1
2020-11-03,107.426949,0.051884,1
...,...,...,...
2025-10-21,262.769989,NaN,0
2025-10-22,258.450012,NaN,0
2025-10-23,259.579987,NaN,0


In [7]:
def get_data(ticker: str) -> pd.DataFrame:
    """
    Fetch historical market data for a given ticker symbol.

    Parameters:
        ticker (str): Ticker symbol of the asset.

    Returns:
        pd.DataFrame: DataFrame containing historical market data.
    """

    data = yf.download(ticker, period="1y", interval="1d")

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    data = data[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

    return data

In [8]:
get_data("AAPL")

[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Volume
Date,,,,,
2024-10-28,232.239171,233.642627,231.472733,232.318787,36087100
2024-10-29,232.020188,233.244486,231.243803,232.587540,35417200
2024-10-30,231.532440,232.388457,228.486618,229.034073,47070900
2024-10-31,228.277598,228.765334,224.325988,224.863495,64370100
2024-11-01,219.946365,224.306080,219.249611,221.877380,65276700
...,...,...,...,...,...
2025-10-21,261.880005,265.290009,261.829987,262.769989,46695900
2025-10-22,262.649994,262.850006,255.429993,258.450012,45015300
2025-10-23,259.940002,260.619995,258.010010,259.579987,32754900
